In [ ]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import metis
tf.get_logger().setLevel('ERROR')

In [ ]:
pip install ampligraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB

In [ ]:

import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [1]:
import numpy as np
import pandas as pd

# Chargement des triplets du jeu de données FB15K
df = pd.read_csv('/content/sample_data/train.txt', sep='\t', header=None, names=['head', 'relation', 'tail'])

# Nombre de partitions souhaitées
k = 12

# Permutation aléatoire des indices des triplets
indices = np.random.permutation(df.index)

# Division des indices en k sous-ensembles
partitions = np.array_split(indices, k)

# Affichage des résultats
for i, part in enumerate(partitions):
    print(f"Partition {i}: {part}")

Partition 0: [135270 441371 417815 ... 362105  55590 188286]
Partition 1: [271755 260857 295085 ... 134638  38575 367773]
Partition 2: [177724  56483 371811 ... 334130  66133 273337]
Partition 3: [407720 234485 245221 ... 367460 474035  97967]
Partition 4: [ 39567 162298 393583 ... 279164 237342 192048]
Partition 5: [ 39515 144227  18215 ... 198349  13110  58682]
Partition 6: [381491  17740 464330 ...  61763  14288 162684]
Partition 7: [255060 430182 192680 ... 216193 386644 376704]
Partition 8: [421724 408010 163775 ... 206965 261252 236475]
Partition 9: [ 78265 255080 192315 ... 167763  62483  49107]
Partition 10: [389137  49711 218398 ... 242556 266836 371366]
Partition 11: [217884  84227  46464 ... 284112 180430 452251]


In [ ]:
import ray

ray.init()

@ray.remote
def my_function(x):
    return x * 2

results = ray.get([my_function.remote(i) for i in range(10)])
print(results)
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# you can continue training from where you left after restoring the model
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./transe_train_logs')

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=5,
                                   k=300,
                                   scoring_type='TransE')

# you can either use optimizers/regularizers/loss/initializers with default values or you can
# import it and customize the hyperparameters and pass it to compile

# Let's create an adam optimizer with customized learning rate =0.005
adam = tf.keras.optimizers.Adam(learning_rate=0.005)
# Let's compile the model with self_advarsarial loss of default parameters
model.compile(optimizer=adam, loss='self_adversarial')

# fit the model to data.
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             callbacks=[tensorboard_callback])

# the training can be visualised using the following command:
# tensorboard --logdir='./transe_train_logs' --port=8891
# open the browser and go to the following URL: http://127.0.0.1:8891/

Epoch 1/10
29/29 [==============================] - 18s 625ms/step - loss: 36581.9766
Epoch 2/10
29/29 [==============================] - 23s 782ms/step - loss: 22528.6367
Epoch 3/10
29/29 [==============================] - 13s 452ms/step - loss: 17284.1270
Epoch 4/10
29/29 [==============================] - 13s 464ms/step - loss: 14601.8623
Epoch 5/10
29/29 [==============================] - 14s 467ms/step - loss: 12988.7441
Epoch 6/10
29/29 [==============================] - 13s 462ms/step - loss: 11921.1309
Epoch 7/10
29/29 [==============================] - 14s 491ms/step - loss: 11167.2803
Epoch 8/10
29/29 [==============================] - 14s 468ms/step - loss: 10608.0420
Epoch 9/10
29/29 [==============================] - 14s 468ms/step - loss: 10177.2725
Epoch 10/10
29/29 [==============================] - 15s 512ms/step - loss: 9836.7529


In [ ]:
pred = model.predict(dataset['test'][:5],
                       batch_size=100)
pred

array([-2.2747788, -2.092898 , -2.3081005, -4.3056355, -4.6757336],
      dtype=float32)

In [ ]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'],
                       batch_size=100,
                       corrupt_side='s,o', # corrupt both subject and object
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@3:', hits_at_n_score(ranks, 3))
print('hits@10:', hits_at_n_score(ranks, 10))
print('hits@5:', hits_at_n_score(ranks, 5))

206/206 [==============================] - 681s 3s/step
MR: 347.97634308640767
MRR: 0.19130546822000338
hits@3: 0.20623348664252864
hits@10: 0.3211664546433115
hits@5: 0.24992660730012722


In [ ]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'],     # test set
                       batch_size=100,      # evaluation batch size
                       corrupt_side='s,o'   # sides to corrupt for scoring and ranking
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

# convert the ranks to milliseconds/step
ranks_ms = ranks * 1000

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@3:', hits_at_n_score(ranks, 3))
print('hits@10:', hits_at_n_score(ranks, 10))
print('hits@5:', hits_at_n_score(ranks, 5))

206/206 [==============================] - 593s 3s/step
MR: 477.0276445836187
MRR: 0.08879523476957402
hits@3: 0.1277032977786476
hits@10: 0.2402632351502104
hits@5: 0.17195909580193758


In [ ]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'],
                       batch_size=100,
                       corrupt_side='o' # corrupt only the object
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@3:', hits_at_n_score(ranks, 3))
print('hits@10:', hits_at_n_score(ranks, 10))
print('hits@5:', hits_at_n_score(ranks, 5))

206/206 [==============================] - 329s 2s/step
MR: 271.89137880418826
MRR: 0.12984180159028205
hits@3: 0.20334670711419905
hits@10: 0.34978960759369804
hits@5: 0.2644583618749388


In [ ]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np

In [ ]:
pip install ampligraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB

In [ ]:
import ampligraph
# load the dataset
from ampligraph.datasets import load_wn18rr
X = load_wn18rr()

In [ ]:
# Import the models from ampligraph.compat
# AmpliGraph 2 APIs support TransE, DistMult, ComplEx, HolE

from ampligraph.compat import DistMult

model = DistMult(batches_count=10, seed=0, epochs=200, k=350, eta=10,
                    # Use adam optimizer with learning rate 1e-3
                    optimizer='adam', optimizer_params={'lr':1e-3},
                    # Use multiclass_nll loss
                    loss='multiclass_nll', loss_params={},
                    # Use L3 regularizer with regularizer weight 1e-3
                    regularizer='LP', regularizer_params={'p':3, 'lambda':1e-3},
                    # Enable stdout messages (set to false if you don't want to display)
                    verbose=True)

In [ ]:
# Create the filter
filter = np.concatenate((X['train'], X['valid'][::10], X['test']))

In [ ]:
# Fit the model on training and validation set
model.fit(X['train'][::2],
          early_stopping = True,
          early_stopping_params = \
                  {
                      'x_valid': X['valid'][::10],  # validation set
                      'criteria':'hits@10',         # Uses hits10 criteria for early stopping
                      'burn_in': 20,                # early stopping kicks in after 100 epochs
                      'check_interval':20,          # validates every 20th epoch
                      'stop_interval':5,            # stops if 5 successive validation checks are bad.
                      'x_filter': filter,           # Use filter for filtering out positives
                      'corruption_entities':'all',  # corrupt using all entities
                      'corrupt_side':'s'            # corrupt only subject
                  }
          )


73 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
4/4 [==============================] - 19s 5s/step

73 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
4/4 [==============================] - 18s 4s/step

73 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
4/4 [==============================] - 17s 4s/step

73 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
4/4 [==============================] - 17s 4s/step

73 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
4/4 [==============================] - 18s 4s/step

73 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
4/4 [==============================] - 17s 4s/step

73 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
4/4 [==============================] - 26s 

In [ ]:
X_test = X['test']
X_test[:2]

array([['06845599', '_member_of_domain_usage', '03754979'],
       ['00789448', '_verb_group', '01062739']], dtype=object)

In [ ]:
# Score assigned to unseen triples
model.predict(X_test[:2])


1 triples containing invalid keys skipped!


array([-0.3586604], dtype=float32)

In [ ]:
# Get embedding of entities
embed = model.get_embeddings(['11647131','02518161'], embedding_type='entity')
print('Embedding size: ', embed.shape[1])
# Notice that the embedding size for ComplEx is double
# compared to the k specified when initializing the model,
# since ComplEx embeddings live in the space of complex numbers.
print('\n Embedding vectors: ')
print(embed)

Embedding size:  350

 Embedding vectors: 
[[ 0.12910752  0.15135074 -0.16565813  0.11561505 -0.04820582  0.19148798
   0.13974549  0.19822668  0.16901898  0.11203824  0.19388743  0.1476582
  -0.2067023  -0.17670077  0.17454751 -0.21035734  0.18862414  0.03869277
  -0.20593068 -0.15297332 -0.17412762  0.18418746  0.07722044  0.08032427
  -0.09058186 -0.16871965 -0.1742022  -0.12594631 -0.21976699  0.16816202
   0.21994954  0.1627388   0.16893199 -0.16982768 -0.1631837  -0.17969781
   0.18598314 -0.19630529 -0.13223849 -0.20012033 -0.15605195 -0.21406525
  -0.18576413 -0.20493993 -0.10127719 -0.2058874   0.20073095  0.18321975
   0.19793671  0.21260428 -0.16347584  0.14388329  0.03916905 -0.20124558
  -0.19271863  0.13540003 -0.2005674   0.13259536  0.14816919 -0.19516236
   0.20309976  0.17451021  0.21679515  0.19556521 -0.19746925  0.16150354
   0.1861482   0.18452062 -0.17845432 -0.18181342  0.20852149  0.19513547
  -0.19364342 -0.18842155  0.08736203  0.15385468 -0.21077445  0.21433

In [ ]:
# get the entity and relation mappings to emb matrix
ent_to_idx, rel_to_idx = model.get_hyperparameter_dict()
len(ent_to_idx), len(rel_to_idx)

(33117, 11)

In [ ]:
# import the evaluate_performance API from compat module
from ampligraph.compat import evaluate_performance
ranks = evaluate_performance(X_test, model, filter_triples=filter, corrupt_side='s,o', verbose=True)

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@3:', hits_at_n_score(ranks, 3))
print('hits@10:', hits_at_n_score(ranks, 10))
print('hits@5:', hits_at_n_score(ranks, 5))


676 triples containing invalid keys skipped!

13473 triples containing invalid keys skipped!
2249/2249 [==============================] - 240s 107ms/step
MR: 9828.000222419929
MRR: 0.2954941397023313
hits@3: 0.3047153024911032
hits@10: 0.3374110320284697
hits@5: 0.3193950177935943


In [ ]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np
import ampligraph

In [ ]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

PATH_TO_DATASET = 'your/path/to/dataset/'

# create the model with transe scoring function
partitioned_model = ScoringBasedEmbeddingModel(eta=2,
                                               k=50,
                                               scoring_type='TransE')
partitioned_model.compile(optimizer='adam', loss='multiclass_nll')

# Here we have specified the path of the input file
# you can also load using default dataloaders load_fb15k_237() and pass numpy array inputs
partitioned_model.fit(PATH_TO_DATASET + 'wn18RR/train.txt',
                      batch_size=10000,
                      partitioning_k=3, # set flag to partition the inputs
                      epochs=10)

FileNotFoundError: ignored

In [ ]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [ ]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# you can continue training from where you left after restoring the model
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./transe_train_logs')

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=5,
                                   k=300,
                                   scoring_type='TransE')

# you can either use optimizers/regularizers/loss/initializers with default values or you can
# import it and customize the hyperparameters and pass it to compile

# Let's create an adam optimizer with customized learning rate =0.005
adam = tf.keras.optimizers.Adam(learning_rate=0.004295)
# Let's compile the model with self_advarsarial loss of default parameters
model.compile(optimizer=adam, loss='self_adversarial')

# fit the model to data.
model.fit(dataset['train'],
             batch_size=1164,
             epochs=10,
             callbacks=[tensorboard_callback])

# the training can be visualised using the following command:
# tensorboard --logdir='./transe_train_logs' --port=8891
# open the browser and go to the following URL: http://127.0.0.1:8891/

Epoch 1/10
235/235 [==============================] - 30s 127ms/step - loss: 2054.4011
Epoch 2/10
235/235 [==============================] - 21s 90ms/step - loss: 1740.3618
Epoch 3/10
235/235 [==============================] - 20s 86ms/step - loss: 1654.8723
Epoch 4/10
235/235 [==============================] - 21s 90ms/step - loss: 1613.1324
Epoch 5/10
235/235 [==============================] - 21s 91ms/step - loss: 1589.1139
Epoch 6/10
235/235 [==============================] - 22s 95ms/step - loss: 1572.5531
Epoch 7/10
235/235 [==============================] - 22s 92ms/step - loss: 1552.8732
Epoch 9/10
235/235 [==============================] - 21s 91ms/step - loss: 1546.3774
Epoch 10/10
235/235 [==============================] - 20s 86ms/step - loss: 1541.0844


In [ ]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'],
                       batch_size=100,
                       corrupt_side='o' # corrupt only the object
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import  hits_at_n_score, mr_score, mrr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks)) #Le rang moyen est la position de classement moyenne des éléments prédits par le modèle parmi tous les éléments possibles.
print('hits@10:', hits_at_n_score(ranks, 10))
print('hits@5:', hits_at_n_score(ranks, 5))
print('hits@3:', hits_at_n_score(ranks, 3))

206/206 [==============================] - 287s 1s/step
MR: 652.9726000587142
MRR: 0.10723492313397588
hits@10: 0.29048830609648696
hits@5: 0.21983560035228497
hits@3: 0.16811821117526177
